In [ ]:
import marimo as mo

In [ ]:
import random
from typing import Self
import pydantic
from ortools.sat.python import cp_model
import didppy

# タレントスケジューリング問題

## 問題

映画の各シーンを撮影する.
各シーンには出演する役者が決まっており,
同じ役者が出演するシーンは同時に撮影することはできない.
各役者には初回撮影日から最終撮影日までの日数分のギャラを支払わねばならない.
間に撮影の無い日があってもその日の報酬も支払われることになる.
この問題では支払うギャラを最小化する.

- $S = \{ 1, \dots, n \}$: 撮影シーン
- $A = \{ 1, \dots, m \}$: 役者
- $A_s \subset A \space (\forall s \in S)$: シーン $s$ を撮るのに必要な役者
- $d_s \in \mathbb{N} \space (\forall s \in S)$: シーン $s$ を撮るのに必要な日数
- $c_a \in \mathbb{N} \space (\forall a \in A)$: 役者 $a$ を 1 日拘束することで発生するギャラ

## 実装

In [ ]:
class Actor(pydantic.BaseModel):
    model_config = pydantic.ConfigDict(frozen=True)
    id: int
    cost: int


class Scene(pydantic.BaseModel):
    model_config = pydantic.ConfigDict(frozen=True)
    id: int
    time: int
    actors: set[int]


class Condition(pydantic.BaseModel):
    model_config = pydantic.ConfigDict(frozen=True)
    actors: list[Actor]
    scenes: list[Scene]

    @staticmethod
    def example(n_scene: int, n_actor: int) -> Self:
        random.seed(0)

        actors = [
            Actor(id=i, cost=random.randint(1, 5)) for i in range(n_actor)
        ]
        scenes = [
            Scene(
                id=j,
                time=random.randint(1, 5),
                actors=random.sample(
                    list(range(n_actor)), random.randint(2, n_actor)
                ),
            )
            for j in range(n_scene)
        ]

        return Condition(actors=actors, scenes=scenes)

In [ ]:
# Toy problem from https://github.com/domain-independent-dp/didp-rs/blob/main/didppy/examples/talent-scheduling.ipynb

_n = 4
_m = 4

_actors = [
    Actor(id=0, cost=1),
    Actor(id=1, cost=3),
    Actor(id=2, cost=1),
    Actor(id=3, cost=2),
]

_scenes = [
    Scene(id=0, time=1, actors={0, 1, 3}),
    Scene(id=1, time=1, actors={1, 2}),
    Scene(id=2, time=1, actors={0, 2, 3}),
    Scene(id=3, time=1, actors={0, 1, 2}),
]

cond1 = Condition(actors=_actors, scenes=_scenes)

### Google OR-Tools でのモデリング

#### 決定変数

- $\text{interval}_s = [\text{start}_s, \text{end}_s) \space (\forall s \in S)$: 各シーンの撮影期間を表す区間変数.
- $\text{keep}_a \space (\forall a \in A)$: 役者 $a$ が拘束された日数

#### 目的関数

\[
    \sum_{a \in A} c_a \text{keep}_a
\]

#### 制約

- $\text{start}_s + d_s = \text{end}_s \space (s \in S)$
- 役者 $a \in A$ が拘束された日数: $\text{keep}_a = \max \{ \text{end}_s \mid s \in S, \space a \in A_s \} - \min \{ \text{start}_s \mid s \in S, \space a \in A_s \}$
- 各役者は同時に 1 つのシーンの撮影しかできない:
  $\text{no-overlap} \{ I_s \mid s \in S, \space a \in A_s \}$
    - × $\to$ どうやら撮影は平行に行えないらしい. 課すべき制約は $\text{no-overlap} \{ I_s \mid s \in S\}$

In [ ]:
class ModelCpSat:
    def __init__(self, cond: Condition):
        self.model = cp_model.CpModel()

        horizon = sum(scene.time for scene in cond.scenes)

        self.starts = [
            self.model.new_int_var(0, horizon, "") for _ in cond.scenes
        ]
        self.intervals = [
            self.model.new_fixed_size_interval_var(
                self.starts[id_scene], scene.time, ""
            )
            for id_scene, scene in enumerate(cond.scenes)
        ]

        # for id_act, actor in enumerate(cond.actors):
        #     self.model.add_no_overlap(
        #         [
        #             self.intervals[id_scene]
        #             for id_scene, scene in enumerate(cond.scenes)
        #             if id_act in scene.actors
        #         ]
        #     )
        self.model.add_no_overlap(self.intervals)

        self.objective = 0
        for id_act, actor in enumerate(cond.actors):
            act_start = self.model.new_int_var(0, horizon, "")
            act_end = self.model.new_int_var(0, horizon, "")
            self.model.add_min_equality(
                act_start,
                [
                    interval.start_expr()
                    for scene, interval in zip(cond.scenes, self.intervals)
                    if id_act in scene.actors
                ],
            )
            self.model.add_max_equality(
                act_end,
                [
                    interval.end_expr()
                    for scene, interval in zip(cond.scenes, self.intervals)
                    if id_act in scene.actors
                ],
            )

            self.objective += actor.cost * (act_end - act_start)

        self.model.minimize(self.objective)

    def solve(self, timeout: int = 180) -> None:
        self.solver = cp_model.CpSolver()
        self.solver.parameters.log_search_progress = True
        self.solver.parameters.max_time_in_seconds = timeout
        self.status = self.solver.solve(self.model)

    def print_solution(self) -> None:
        shoots = [
            (f"Shoot {i}", self.solver.value(interval.start_expr()))
            for i, interval in enumerate(self.intervals)
        ]
        shoots.sort(key=lambda x: x[1])

        for name, start in shoots:
            print(f"{name} starts at {start}")

In [ ]:
model1 = ModelCpSat(cond1)
model1.solve()

mo.md(f"Time: {model1.solver.wall_time:.9f}")


Starting CP-SAT solver v9.13.4784
Parameters: max_time_in_seconds: 180 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x4d9da0179b3d1bee)
#Variables: 12 (#ints: 8 in objective) (4 primary variables)
  - 12 in [0,4]
#kInterval: 4
#kLinMax: 8 (#expressions: 22)
#kNoOverlap: 1 (#intervals: 4)

Starting presolve at 0.00s
  1.39e-05s  0.00e+00d  [DetectDominanceRelations] 
  1.58e-04s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  4.78e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  3.24e-06s  0.00e+00d  [DetectDuplicateColumns] 
  9.51e-06s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 47 nodes and 60 arcs.
[Symmetry] Symmetry computation done. time: 1.7273e-05 dtime: 1.008e-05
  9.04e-06s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  1.18e-04s  1.08e-07d  [Probe] 
  1.89e-06s  0.00e+00d  [MaxClique] 
  8.31e-06s  0.00e+00d  [DetectDominanceRelati

In [ ]:
model1.print_solution()

Shoot 2 starts at 0
Shoot 0 starts at 1
Shoot 3 starts at 2
Shoot 1 starts at 3


### didp でのモデリング

#### DP Formulation

シーンの撮影が途中まで完了し, 残りが $Q \subset S$ であるとする.
次に $s \in Q$ を撮る場合, 拘束される役者の集合 $L(s, Q)$ は

\[
    L(s, Q) = A_s \cup \left( \bigcup_{s' \in S \setminus Q} A_{s'} \cap \bigcup_{s' \in Q} A_{s'} \right)
\]

となる.
右辺の大括弧内は今までのシーン $S \setminus Q$ の撮影で既に呼び寄せていて,
この先も撮影があるため解放できない役者の集合を表す.
シーン $s$ の撮影が終わると支払われるギャラが $d_s \sum_{a \in L(s, Q)} c_a$ だけ増加し,
$Q$ が $Q \setminus \{ s \}$ で更新される.

\begin{align*}
    &\text{compute} & &V(S) \\
    &\text{s.t.} & &V(Q) = \begin{dcases}
            \min_{s \in Q} \left( d_s \sum_{a \in L(s, Q)} c_a + V(Q \setminus \{s\}) \right) & \text{if } Q \neq \emptyset \\
            0 & \text{if } Q = \emptyset
        \end{dcases}
\end{align*}

$V(Q)$ は $Q$ だけを撮影するコストを表していることに注意.

#### Force Transition

シーン $s \in Q$ に必要な役者がすでに現場に拘束されておりかつ,
$s$ がその全員を必要とする撮影の場合, つまり

\[
    A_s = \bigcup_{s' \in S \setminus Q} A_{s'} \cap \bigcup_{s' \in Q} A_{s'} \quad (s \in Q)
\]

のとき, シーン $s$ を直ちに撮影するのが最適となる.
この場合, 上記の更新規則より高い優先順位で下記の更新を行う.

\[
    V(Q) = d_s \sum_{a \in A_s} c_a + V(Q \setminus \{ s \})
\]

#### Dual Bound

ドメイン知識で計算の高速化ができるっぽい.
今回のケースだと下記のような制約を入れる.

\[
    V(Q) \geq \sum_{s \in Q} d_s \sum_{a \in A_s} c_a
\]

In [ ]:
class ModelDidp:
    def __init__(self, cond: Condition):
        self.model: didppy.Model = didppy.Model()

        scene_indices: list[int] = list(range(len(cond.scenes)))

        # S, A
        objtype_scene: didppy.ObjectType = self.model.add_object_type(
            number=len(cond.scenes)
        )
        objtype_actor: didppy.ObjectType = self.model.add_object_type(
            number=len(cond.actors)
        )

        # Q
        remaining: didppy.SetVar = self.model.add_set_var(
            object_type=objtype_scene,
            target=scene_indices,
        )

        # 各シーンの撮影に必要な役者の集合のリスト
        # didp の example では object_type が scene だったが多分 actor が正しい.
        scene_to_actors_table: didppy.SetTable1D = self.model.add_set_table(
            [scene.actors for scene in cond.scenes], object_type=objtype_actor
        )

        # 各役者の日ごとのギャラ
        actor_to_cost: didppy.IntTable1D = self.model.add_int_table(
            [actor.cost for actor in cond.actors]
        )

        # 既に来てもらった役者の集合. 現地に留まっているとは限らず, 帰ってる可能性もある.
        # 必須役者集合のリスト scene_to_actors_table に remaining の補集合を入れることで求める
        came_to_location: didppy.SetExpr = scene_to_actors_table.union(
            remaining.complement()
        )

        # これから撮影のある役者
        # 必須役者集合のリスト scene_to_actors_table に remaining を入れることで求める
        standby: didppy.SetExpr = scene_to_actors_table.union(remaining)

        # 既に来てもらったことのある役者とこれから撮影のある役者の共通部分.
        # 次の撮影時にギャラを支払う必要がある.
        # Define a state function to avoid redundant evaluation of an expensive expression
        on_location: didppy.SetExpr = self.model.add_set_state_fun(
            came_to_location & standby
        )

        # Base Case
        self.model.add_base_case(conditions=[remaining.is_empty()], cost=0)

        # Transition
        for s in scene_indices:
            # 残ってもらっている役者と s の撮影に必要な役者の和集合.
            # 彼らに対してのみギャラを支払う.
            on_location_s: didppy.SetExpr = (
                scene_to_actors_table[s] | on_location
            )

            shoot: didppy.Transition = didppy.Transition(
                name=f"shoot {s}",
                cost=cond.scenes[s].time * actor_to_cost[on_location_s]
                + didppy.IntExpr.state_cost(),
                effects=[(remaining, remaining.remove(s))],
                preconditions=[remaining.contains(s)],
            )
            self.model.add_transition(shoot)

        # Dual Bound: 各シーンを撮影する最低コスト
        scene_to_min_cost: didppy.IntTable1D = self.model.add_int_table(
            [
                cond.scenes[s].time
                * sum(cond.actors[a].cost for a in cond.scenes[s].actors)
                for s in scene_indices
            ]
        )
        self.model.add_dual_bound(scene_to_min_cost[remaining])

        # Force Transition
        for s in scene_indices:
            shoot: didppy.Transition = didppy.Transition(
                name=f"forced shoot {s}",
                cost=scene_to_min_cost[s] + didppy.IntExpr.state_cost(),
                effects=[(remaining, remaining.remove(s))],
                preconditions=[
                    remaining.contains(s),
                    scene_to_actors_table[s] == on_location,
                ],
            )
            self.model.add_transition(shoot, forced=True)

    def solve(self, timeout=180, threads: int = 8) -> None:
        self.solver: didppy.CABS = didppy.CABS(
            self.model, threads=threads, quiet=False, time_limit=timeout
        )
        self.solution: didppy.Solution = self.solver.search()

    def print_solution(self) -> None:
        print("Transitions to apply:")
        print("")

        for _t in self.solution.transitions:
            print(_t.name)

        print()
        print(f"Cost: {self.solution.cost}")

In [ ]:
model2 = ModelDidp(cond1)
model2.solve()
mo.md(f"Time: {model2.solution.time:.9f}")

Solver: CABS from DIDPPy v0.9.0
Searched with beam size: 1, threads: 8, kept: 10, sent: 0
Searched with beam size: 1, expanded: 4, elapsed time: 0.005840198
New primal bound: 24, expanded: 4, elapsed time: 0.005850627
Searched with beam size: 2, threads: 8, kept: 9, sent: 7
Searched with beam size: 2, expanded: 11, elapsed time: 0.005952541
New dual bound: 19, expanded: 11, elapsed time: 0.005953803
New primal bound: 20, expanded: 11, elapsed time: 0.005956048
Searched with beam size: 4, threads: 8, kept: 2, sent: 3
Searched with beam size: 4, expanded: 16, elapsed time: 0.006056217
Searched with beam size: 8, threads: 8, kept: 0, sent: 5
Searched with beam size: 8, expanded: 21, elapsed time: 0.006309398
Searched with beam size: 16, threads: 8, kept: 0, sent: 5
Searched with beam size: 16, expanded: 27, elapsed time: 0.006506171


In [ ]:
model2.print_solution()

Transitions to apply:

shoot 1
shoot 3
shoot 0
forced shoot 2

Cost: 20


In [ ]:
cond2 = Condition.example(n_scene=20, n_actor=10)

cond2.model_dump()

In [ ]:
model3 = ModelCpSat(cond2)
model3.solve()
mo.md(f"Time: {model3.solver.wall_time:.9f}")


Starting CP-SAT solver v9.13.4784
Parameters: max_time_in_seconds: 180 log_search_progress: true
Setting number of workers to 12

Initial optimization model '': (model_fingerprint: 0x41b4b1f1b6986984)
#Variables: 40 (#ints: 20 in objective) (20 primary variables)
  - 40 in [0,52]
#kInterval: 20
#kLinMax: 20 (#expressions: 220)
#kNoOverlap: 1 (#intervals: 20)

Starting presolve at 0.00s
  4.26e-05s  0.00e+00d  [DetectDominanceRelations] 
  5.98e-04s  0.00e+00d  [PresolveToFixPoint] #num_loops=2 #num_dual_strengthening=1 
  2.08e-06s  0.00e+00d  [ExtractEncodingFromLinear] 
  1.91e-05s  0.00e+00d  [DetectDuplicateColumns] 
  1.76e-04s  0.00e+00d  [DetectDuplicateConstraints] 
[Symmetry] Graph for symmetry has 301 nodes and 500 arcs.
[Symmetry] Symmetry computation done. time: 4.3352e-05 dtime: 6.596e-05
  6.58e-05s  0.00e+00d  [DetectDuplicateConstraintsWithDifferentEnforcements] 
  2.96e-04s  3.00e-06d  [Probe] #new_bounds=20 
  2.37e-06s  0.00e+00d  [MaxClique] 
  3.52e-05s  0.00e+00d

#5       0.01s best:1456  next:[469,1455] no_lp
#6       0.01s best:1447  next:[469,1446] no_lp
#7       0.01s best:1445  next:[469,1444] no_lp
#Bound   0.01s best:1445  next:[473,1444] bool_core (num_cores=2 [size:2 mw:4 d:1] a=18 d=1 fixed=12/84 clauses=13)
#Bound   0.01s best:1445  next:[477,1444] bool_core (num_cores=2 [cover] a=18 d=1 fixed=12/87 clauses=14)
#Bound   0.01s best:1445  next:[481,1444] bool_core (num_cores=2 [cover] a=18 d=1 fixed=13/90 clauses=16)
#Bound   0.01s best:1445  next:[485,1444] bool_core (num_cores=2 [cover] a=18 d=1 fixed=14/93 clauses=19)
#Bound   0.01s best:1445  next:[489,1444] bool_core (num_cores=2 [cover] a=18 d=1 fixed=15/96 clauses=23)
#Bound   0.01s best:1445  next:[493,1444] bool_core (num_cores=2 [cover] a=18 d=1 fixed=16/99 clauses=28)
#Bound   0.01s best:1445  next:[497,1444] bool_core (num_cores=2 [cover] a=18 d=1 fixed=17/102 clauses=34)
#Bound   0.01s best:1445  next:[501,1444] bool_core (num_cores=2 [cover] a=18 d=1 fixed=18/105 clauses=

#11      0.03s best:1420  next:[773,1419] no_lp


#12      0.05s best:1415  next:[797,1414] no_lp


#13      0.07s best:1412  next:[797,1411] no_lp


...

#Bound  44.50s best:1366  next:[1363,1365] bool_core (num_cores=61 [size:1 mw:1] a=1 d=9 fixed=328/1380 clauses=75'719)


#Bound  45.49s best:1366  next:[1364,1365] bool_core (num_cores=62 [size:1 mw:1] a=1 d=9 fixed=333/1386 clauses=75'036)


#Bound  48.67s best:1366  next:[1365,1365] bool_core (num_cores=63 [size:1 mw:1] a=1 d=9 fixed=334/1392 clauses=89'324)


#Done   52.94s core
#Done   52.94s quick_restart



Task timing                             n [     min,      max]      avg      dev     time         n [     min,      max]      avg      dev    dtime
                        'core':         1 [  52.94s,   52.94s]   52.94s   0.00ns   52.94s         1 [   1.36m,    1.36m]    1.36m   0.00ns    1.36m
                  'default_lp':         1 [  52.95s,   52.95s]   52.95s   0.00ns   52.95s         1 [  19.01s,   19.01s]   19.01s   0.00ns   19.01s
            'feasibility_pump':       177 [ 38.17us,   1.31ms]  86.14us 107.21us  15.25ms       176 [ 16.73us,   1.84ms]  27.08us 136.93us   4.77ms
                       'fixed':         1 [  52.95s,   52.95s]   52.95s   0.00ns   52.95s         1 [  39.71s,   39.71s]   39.71s   0.00ns   39.71s
                          'fj':         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns         0 [  0.00ns,   0.00ns]   0.00ns   0.00ns   0.00ns
                          'fj':         1 [ 25.35ms,  25.35ms]  25.35ms   0.00ns  25.35ms         1 [ 46.05ms, 

In [ ]:
model3.print_solution()

Shoot 18 starts at 0
Shoot 5 starts at 1
Shoot 13 starts at 2
Shoot 1 starts at 5
Shoot 12 starts at 10
Shoot 15 starts at 12
Shoot 9 starts at 14
Shoot 17 starts at 15
Shoot 2 starts at 17
Shoot 11 starts at 20
Shoot 3 starts at 23
Shoot 7 starts at 27
Shoot 4 starts at 29
Shoot 10 starts at 31
Shoot 14 starts at 36
Shoot 6 starts at 38
Shoot 16 starts at 39
Shoot 0 starts at 40
Shoot 8 starts at 45
Shoot 19 starts at 47


In [ ]:
model4 = ModelDidp(cond2)
model4.solve(threads=10)
mo.md(f"Time: {model4.solution.time:.9f}")

Solver: CABS from DIDPPy v0.9.0
Searched with beam size: 1, threads: 10, kept: 192, sent: 0
Searched with beam size: 1, expanded: 20, elapsed time: 0.000178639
New primal bound: 1599, expanded: 20, elapsed time: 0.000180753
Searched with beam size: 2, threads: 10, kept: 173, sent: 173
Searched with beam size: 2, expanded: 58, elapsed time: 0.000428313
New dual bound: 1112, expanded: 58, elapsed time: 0.000429255
New primal bound: 1526, expanded: 58, elapsed time: 0.000436098
Searched with beam size: 4, threads: 10, kept: 160, sent: 524
Searched with beam size: 4, expanded: 127, elapsed time: 0.000712452
Searched with beam size: 8, threads: 10, kept: 177, sent: 1165
Searched with beam size: 8, expanded: 271, elapsed time: 0.001207581
New primal bound: 1471, expanded: 271, elapsed time: 0.001219223
Searched with beam size: 16, threads: 10, kept: 230, sent: 2313
Searched with beam size: 16, expanded: 553, elapsed time: 0.004337678
New primal bound: 1448, expanded: 553, elapsed time: 0.004

In [ ]:
model4.print_solution()

Transitions to apply:

shoot 19
shoot 0
shoot 8
shoot 16
shoot 14
shoot 6
shoot 10
shoot 4
shoot 11
forced shoot 3
forced shoot 7
shoot 2
shoot 17
shoot 15
shoot 12
shoot 9
shoot 1
shoot 13
shoot 5
forced shoot 18

Cost: 1366
